In [1]:
# # Skriptas zive EKG pūpsnių CNN VU klasifikatoriaus testavimui ir tikslumo įvertinimui.
# Įvertinimas atliekamas su rpeaks, paimtais iš json failo, kuris yra gydytojų koreguotas
# Toks įvertinimas parodo, ką galėtų pasiekti ML klasifikacija, jei Neurokit be klaidų
# sužymėtų rpeaks
#  
# Skripte yra galimybė išvesti ekstrasistolių vietas įraše.
# Dirbant su daug įrašų reiktų užblokuoti: classification = []  # Užblokuota
 
import tensorflow as tf
import pickle
import pandas as pd
import numpy as np
import neurokit2 as nk
import time
import sys, os, json
from pathlib import Path
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_recall_fscore_support

from vertinimas_util import show_confusion_matrix
from vertinimas_util import create_dir, zive_read_file_1ch, zive_read_df_rpeaks
from vertinimas_util import runtime
from vertinimas_util import get_label_sums, get_rid_off_class_3

from zive_cnn_fda_vu_v3_micro_modif import classify_cnn_fda_vu_vasara_v2_modif

np.set_printoptions(threshold=sys.maxsize)

print(tf.__version__)


def zive_read_rec_id(db_path, file_name):
    file_path = Path(db_path, file_name + '.json')
    with open(file_path,'r', encoding="utf8") as f:
        data = json.loads(f.read())
    userId = data["userId"]
    recId = data["recordingId"]
    return userId, recId


2.6.0


In [2]:
# Pagrindinis skriptas

print("Skriptas zive-arrh EKG segmentų apmokyto klasifikatoriaus tikslumo įvertinimui")
print('Modelis CNN VU su EKG sekos reikšmėmis, EKG formos požymiais, RR intervalais prieš ir po R dantelio')

my_os=sys.platform
print("OS in my system : ",my_os)

if my_os != 'linux':
    OS = 'Windows'
else:  
    OS = 'Ubuntu'

# Pasiruošimas

# //////////////// NURODOMI PARAMETRAI /////////////////////////////////////////////////////

# Bendras duomenų aplankas, kuriame patalpintas subfolderis name_db

if OS == 'Windows':
    Duomenu_aplankas = 'D:\\DI'   # variantas: Windows
    # Duomenu_aplankas = 'F:\DI\Data\MIT&ZIVE\VU'   # variantas: Herkulis
else:
    Duomenu_aplankas = '/home/kesju/DI'   # arba variantas: UBUNTU, be Docker
    # Duomenu_aplankas = '/home/kesju/DI/GITLAB'   # arba variantas: UBUNTU, be Docker

# jei variantas Docker pasirenkame:
# Duomenu_aplankas = '/Data/MIT&ZIVE'

# Vietinės talpyklos aplankas ir pūpsnių atributų failas
db_folder = 'DUOM_2022_RUDUO_2'
# db_folder = 'analysis'

# Failai pūpsnių klasių formavimui
selected_beats = {'N':0, 'S':1, 'V':2}
all_beats =  {'N':0, 'S':1, 'V':2, 'U':3}  

# Diskretizavimo dažnis:
fs = 200

# /////////////////////////////////////////////////////////////////

#  Nuoroda į aplanką su MIT2ZIVE duomenų rinkiniu
db_path = Path(Duomenu_aplankas, db_folder)

# Nuoroda į aplanką su EKG įrašais (.npy) ir anotacijomis (.json)
rec_dir = Path(db_path, 'records_selected')
# rec_dir = Path(db_path, 'test')

# Nuoroda į modelio aplanką
# model_dir = Path(Duomenu_aplankas, 'DNN', 'best_models', 'all_ft')
model_dir = 'model_cnn_fda_vu_v1'

# Užduodame, ar filtruojame įrašus
Filtr_flag = False  # True - filtruoti


# Išvedame parametrus
print("\nBendras duomenų aplankas: ", Duomenu_aplankas)
print("Zive duomenų aplankas: ", db_folder)
print("Aplankas su originaliais EKG įrašais ir anotacijomis (.json) ", rec_dir)
print("Diskretizavimo dažnis: ", fs)
print('Klasifikavimo schema:', selected_beats)
print('Klasių skaičius:', len(selected_beats))
print('Visos galimos anotacijos:', list(all_beats.keys()))
print("Modelio ir scaler parametrai nuskaitomas iš aplanko: ", model_dir)
# print("\n")

if (Filtr_flag):
    # Filtruojame izoliniją su 0.5 Hz žemo dažnumo filtru
    lowcut = 0.5
    method = 'butterworth'
    order = 5
    print(f"\nEKG įrašai filtruojami su Neurokit2")
    print(f"Parametrai: lowcut:{lowcut} method: {method} order: {order} ")
else:
    print(f"\nEKG įrašas nefiltruotas")
print()

# PASIRUOŠIMAS

# pd.set_option("display.max_rows", 6000)
# pd.set_option("display.max_columns",200)
# pd.set_option('display.width', 1000)

import warnings
warnings.filterwarnings("ignore")

# Naudojamų požymių sąrašas 
all_features = ['seq_size','RR_l_0', 'RR_r_0', 'RR_r/RR_l','wl_side','wr_side',
                'signal_mean', 'signal_std', 'P_val', 'Q_val', 'R_val', 'S_val', 'T_val',
                'P_pos', 'Q_pos', 'R_pos', 'S_pos', 'T_pos', 'QRS', 'PR', 'ST', 'QT', '0', '1', '2',
                '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18',
                '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32',
                '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46',
                '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60',
                '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74',
                '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88',
                '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102',
                '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114',
                '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126',
                '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138',
                '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150',
                '151', '152', '153', '154', '155', '156', '157', '158', '159', '160', '161', '162',
                '163', '164', '165', '166', '167', '168', '169', '170', '171', '172', '173', '174',
                '175', '176', '177', '178', '179', '180', '181', '182', '183', '184', '185', '186',
                '187', '188', '189', '190', '191', '192', '193', '194', '195', '196', '197', '198',
                '199']

print(f"Atliekama pūpsnių su anotacijomis {list(selected_beats.keys())} pacientų įrašuose klasifikacija")

# NURODOME PACIENTŲ SĄRAŠĄ
# VARIANTAS, KAI UŽDUODAMI FAILŲ VARDAI IR SKAITOMI ORIGINALUS ZIVE FAILAI 
# 20 atrinktų testinių sąrašas
FileNames = [
1626934.963,
1626931.201,
1630715.664,
1630714.569,
1630729.576,
1630735.143,
1630693.635,
1630734.526,
1630718.396,
1630721.49,
1631139.883,
1631083.411,
1631039.923,
1631029.786,
1632342.032,
1633428.56,
1633584.898,
1633405.853,
1634112.089,
1636451.86
]

FileNames = [1626934.963] #

# 4 testiniai įrašai GitLab
FileNames = [1625400.796, 1625402.027, 1630757.924, 1631141.764] #  

FileNames = [1625400.796] #
FileNames = [1625402.027] #
FileNames = [1630757.924] #
FileNames = [1631141.764] #
# mark1

path_for_results = Path(db_path, 'rezultatai_tst')
print(f"\nAplankas rezultatams: {path_for_results}")
create_dir(path_for_results)

# Kas kiek išvedamas apdorotų sekų skaičius
show_period = 100

# Klasių simbolinių vardų sąrašas ir klasių skaičius
class_names = list(selected_beats.keys()) 
n_classes = len(selected_beats)
# print(class_names)

                    # SUFORMUOJAME all_beats_attr

# Failai pūpsnių klasių formavimui
annot_grouping = {'N':'N', 'S':'S', 'V':'V', 'U':'U'}

# Sukūriame masyvą sekų atributų sąrašui
# https://sparkbyexamples.com/pandas/pandas-empty-dataframe-with-specific-column-types/

all_beats_attr = pd.DataFrame({'file_name': pd.Series(dtype='str'),
                   'sample': pd.Series(dtype='int'),
                   'symbol': pd.Series(dtype='str'),
                   'label': pd.Series(dtype='int'),
                   'pred_label': pd.Series(dtype='int')
                   })

df_rec_results = pd.DataFrame({'file_name':pd.Series(dtype='str'),  
    'userId':pd.Series(dtype='str'), 'recId':pd.Series(dtype='int'), 'signal_length':pd.Series(dtype='int'),
    'N':pd.Series(dtype='int'), 'S':pd.Series(dtype='int'), 'V':pd.Series(dtype='int'), 'U':pd.Series(dtype='int'),
    'Nml':pd.Series(dtype='int'), 'Sml':pd.Series(dtype='int'), 'Vml':pd.Series(dtype='int'), 'Uml':pd.Series(dtype='int'),
    'Nprec':pd.Series(dtype='float') , 'Nrec':pd.Series(dtype='float'), 'Nfsc':pd.Series(dtype='float'),
    'Sprec':pd.Series(dtype='float') , 'Srec':pd.Series(dtype='float'), 'Sfsc':pd.Series(dtype='float'),
    'Vprec':pd.Series(dtype='float') , 'Vrec':pd.Series(dtype='float'), 'Vfsc':pd.Series(dtype='float'), 
     'Err%':pd.Series(dtype='float'), 'Noise%':pd.Series(dtype='float')})
# Pandas Empty DataFrame with Specific Column Types
# https://sparkbyexamples.com/pandas/pandas-empty-dataframe-with-specific-column-types/

rows_list = []

# CIKLAS PER PACIENTŲ ĮRAŠUS

start_time = time.time()

file_names=["%.3f" % i for i in FileNames]
print(file_names)

# Ciklas per visą įrašų sąrašą
for file_name in file_names:
    print(f"\nZive įrašas:  {file_name:>2}")
    
    df_rpeaks = zive_read_df_rpeaks(rec_dir, file_name)
    atr_sample = df_rpeaks['sampleIndex'].to_numpy()
    atr_symbol = df_rpeaks['annotationValue'].to_numpy()

    # Nuskaitome EKG įrašą (Zive formatu)
    filepath = Path(rec_dir, file_name)
    sign_raw = zive_read_file_1ch(filepath)

    signal_length = sign_raw.shape[0]
    signal = sign_raw
    
    if (Filtr_flag):
        signal = nk.signal_filter(signal=sign_raw, sampling_rate=200, lowcut=lowcut, method=method, order=order)
        # signal = nk.signal_filter(signal=sign_raw, sampling_rate=200, lowcut=0.2, method="butterworth", order=5)
        signal_length = signal.shape[0]
    else:
        signal = sign_raw
        signal_length = sign_raw.shape[0]

    # Surandame ir išvedame įrašo atributus
    userId, recId = zive_read_rec_id(rec_dir, file_name)

    print(f"\nfile_name: {file_name:>2} userId: {userId} recId: {recId} signal_length: {signal_length}")

    # Jei pasitaiko symbol 'U' arba 'F', pūpsniui suteikiame klasę 3, kurią vėliau apvalysime  
    test_labels = np.array([all_beats[symbol] for symbol in atr_symbol])

    label_sums, total = get_label_sums(test_labels, all_beats)  
    # print("test_labels: ", list(all_beats.keys()), label_sums, "Total:", total)

    # Surandame ML anotacijų skaitmenines reikšmes pred_labels
    # Jei atr_symbol atranda anotaciją 'U', tai jos neklasifikuoja,
    # bet iš karto patalpina '3' į pred_labels atitinkamą vietą.
    # Į pred_labels taip pat įrašomas '3' pirmam ir paskutiniam pūpsniui,
    # o taip pat pakliuvusiems į ommited sritį, 
    # pred_labels = predict_cnn_fda_vu_v1_micro(signal, atr_sample, atr_symbol, model_dir)
    # mark2      Taisomas vieta //////////////////////////////////////////////

    pred_labels = classify_cnn_fda_vu_vasara_v2_modif(signal, atr_sample, model_dir)
    
    # pred_labels turi būti tokio pat ilgio, kaip ir test_labels
    if (len(test_labels) != len(pred_labels)):
        raise Exception(f"Klaida! file_name: {file_name}. Nesutampa test_labels ir pred_labels ilgiai")     

    label_sums_ml, total = get_label_sums(pred_labels, all_beats)  
    # print("pred_labels: ", list(all_beats.keys()), label_sums_ml, "Total:", total)

    # Surandame vietas su ekstrasistolemis ir išvedame jų sąrašą vizualiniam įvertinimui. 
    classification=[]
    for i, i_sample in enumerate(atr_sample):
        if ((pred_labels[i] != 0) or test_labels[i] != 0):
            classification.append({'i':i, 'sample':i_sample, 'annot':test_labels[i], 'pred':pred_labels[i]})

    # Vietų sąrašas išvedamas
    # Dirbant su daug įrašų sąrašo išvedimą reikia užblokuoti !!!!!!!!!!!!!!!!!!!!!!!!!!!
    classification = []  # uzblokuota
    if (classification):
        print('\nVietos su ekstrasistolėmis test_y arba pred_y:')
        for row in classification:
            print(f"i: {row['i']:>7} sample: {row['sample']:>7}   annot_label: {row['annot']:>2}   pred_label: {row['pred']:>2}")  

    # Ciklas per visas paciento įrašo anotacijas (simbolius) ir jų vietas (i_sample)
    for i, i_sample in enumerate(atr_sample):

        # Formuojame pūpsnio atributus
        beats_attr = {'file_name':file_name, 'sample':int(i_sample), 
                        'symbol':str(atr_symbol[i]), 'label':test_labels[i], 'pred_label':pred_labels[i]}

        # Kaupiame su concat
        df_new_row = pd.DataFrame([beats_attr])
        all_beats_attr = pd.concat([all_beats_attr, df_new_row])

 # Suformuojame klasių numerių masyvus confusion matricai skaičiuoti, surandama confusion matrica

    # pred_labels turi būti tokio pat ilgio, kaip ir test_labels
    if (len(test_labels) != len(pred_labels)):
        raise Exception(f"Klaida! file_name: {file_name}. Nesutampa test_labels ir pred_labels ilgiai")     

    test_labels_mod, pred_labels_mod = get_rid_off_class_3(test_labels, pred_labels)
    label_sums_ml_3, total = get_label_sums(pred_labels_mod, all_beats)  
    # print("pred_labels_3: ", list(all_beats.keys()), label_sums_ml_3, "Total:", total)

    confusion = confusion_matrix(test_labels_mod, pred_labels_mod)
    # print()
    # print(confusion)
    prec,rec,fsc,sup = precision_recall_fscore_support(test_labels_mod, pred_labels_mod, labels=[0, 1, 2], zero_division=0)

    str1 =f"N:{int(label_sums[0]):>5} S:{(int(label_sums[1])):3} V:{int(label_sums[2]):3} U:{int(label_sums[3]):3}" 
    str2 =f"  Nml:{int(label_sums_ml[0]):>5} Sml:{(int(label_sums_ml[1])):3} Vml:{int(label_sums_ml[2]):3} Uml:{int(label_sums_ml[3]):3}" 
    str3 = f"  Nprec:{prec[0]:>5.2f} Nrec:{rec[0]:5.2f} Nfsc:{fsc[0]:5.2f}"
    str4 = f"  Sprec:{prec[1]:>5.2f} Srec:{rec[1]:5.2f} Sfsc:{fsc[1]:5.2f}"
    str5 = f"  Vprec:{prec[2]:>5.2f} Vrec:{rec[2]:5.2f} Vfsc:{fsc[2]:5.2f}"
    # print()
    print(str1+str2+str3+str4+str5)

    dict_rec_results = {'file_name':file_name,
    'userId': userId, 'recId': recId, 'signal_length': signal_length,
    'N':label_sums[0], 'S':label_sums[1], 'V':label_sums[2], 'U':label_sums[3],
    'Nml':label_sums_ml[0], 'Sml':label_sums_ml[1], 'Vml':label_sums_ml[2], 'Uml':label_sums_ml[3],
    'Nprec':prec[0], 'Nrec':rec[0], 'Nfsc':fsc[0],
    'Sprec':prec[1], 'Srec':rec[1], 'Sfsc':fsc[1],
    'Vprec':prec[2], 'Vrec':rec[2], 'Vfsc':fsc[2] 
    }
    rows_list.append(dict_rec_results)

df_rec_results =  pd.DataFrame(rows_list) 

# Ciklo per pacientų įrašus pabaiga

end_time = time.time()
print('\n')
runtime(end_time-start_time)

filepath = Path(path_for_results, 'klasifikacijos_rezultatai_irasams.csv') 
df_rec_results.to_csv(filepath)    
print(f'\nRezultatai įrašyti į:  {filepath}')

# Pernumeruojame indeksus, kad būtų nuo 0 iš eilės
all_beats_attr.reset_index(inplace = True, drop = True)

# Įrašome sekos atributų masyvą į rec_dir aplanką
file_path = Path(path_for_results, 'all_beats_attr.csv')
all_beats_attr.to_csv(file_path)
print("\nAtributų freimas įrašytas: į ", file_path, "\n" )


Skriptas zive-arrh EKG segmentų apmokyto klasifikatoriaus tikslumo įvertinimui
Modelis CNN VU su EKG sekos reikšmėmis, EKG formos požymiais, RR intervalais prieš ir po R dantelio
OS in my system :  linux

Bendras duomenų aplankas:  /home/kesju/DI
Zive duomenų aplankas:  DUOM_2022_RUDUO_2
Aplankas su originaliais EKG įrašais ir anotacijomis (.json)  /home/kesju/DI/DUOM_2022_RUDUO_2/records_selected
Diskretizavimo dažnis:  200
Klasifikavimo schema: {'N': 0, 'S': 1, 'V': 2}
Klasių skaičius: 3
Visos galimos anotacijos: ['N', 'S', 'V', 'U']
Modelio ir scaler parametrai nuskaitomas iš aplanko:  model_cnn_fda_vu_v1

EKG įrašas nefiltruotas

Atliekama pūpsnių su anotacijomis ['N', 'S', 'V'] pacientų įrašuose klasifikacija

Aplankas rezultatams: /home/kesju/DI/DUOM_2022_RUDUO_2/rezultatai_tst
Directory '/home/kesju/DI/DUOM_2022_RUDUO_2/rezultatai_tst' already exists
['1631141.764']

Zive įrašas:  1631141.764

file_name: 1631141.764 userId: 613b1d673d08d4d1f3cdc8f8 recId: 613b22d53d08d4fe94cdc9e

2023-02-18 20:37:22.286173: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-18 20:37:22.292131: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2023-02-18 20:37:23.074042: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


pred_y: [0 1 2 3] [805  43  15   2] 865
N:  845 S: 20 V:  0 U:  0  Nml:  805 Sml: 43 Vml: 15 Uml:  2  Nprec: 1.00 Nrec: 0.95 Nfsc: 0.97  Sprec: 0.12 Srec: 0.25 Sfsc: 0.16  Vprec: 0.00 Vrec: 0.00 Vfsc: 0.00


Runtime: 00:00:38

Rezultatai įrašyti į:  /home/kesju/DI/DUOM_2022_RUDUO_2/rezultatai_tst/klasifikacijos_rezultatai_irasams.csv

Atributų freimas įrašytas: į  /home/kesju/DI/DUOM_2022_RUDUO_2/rezultatai_tst/all_beats_attr.csv 



In [3]:
# MODELIO TIKSLUMO VERTINIMO IŠ VERTINIMO IMTIES REZULTATAI

# Nuskaitome pūpsnių atributų masyvą
file_path = Path(path_for_results, 'all_beats_attr.csv')
all_beats_attr = pd.read_csv(file_path, index_col=0, dtype = {'file_name': str, 
                                            'sample': int, 'symbol': str, 'label': int, 'pred_label':int })

# Sukūriame anotuotų ir automatiškai priskirtų klasių visų įrašų pūpsniams sąrašus 
validate_ind_lst = all_beats_attr.index
y_validate = np.array(all_beats_attr['label']).astype('int')
y_predicted = np.array(all_beats_attr['pred_label']).astype('int')

print("\nMODELIO TIKSLUMO VERTINIMO REZULTATAI")
print("Modelis iš aplanko: ", model_dir)

label_sums, total = get_label_sums(y_validate, all_beats)  
print("Pūpsnių klasės: ", list(all_beats.keys()), label_sums, "Total:", total)
print("Klasifikuojamos klasės: ['N', 'S', 'V']")
y_validate_mod, y_predicted_mod = get_rid_off_class_3(y_validate, y_predicted)

# APIBENDRINTI REZULTATAI

print('\nAPIBENDRINTI REZULTATAI\n')

# +++++++++++++++++++++++++++++++++++  čia reiktų įdėtiy_validate, y_predicted valymą nuo 3 klasės

# Skaičiuojame ir išvedame klasifikavimo lentelę
confusion = confusion_matrix(y_validate_mod, y_predicted_mod)
pd.set_option('display.precision',3)
show_confusion_matrix(confusion, class_names)
# print('\n')

print("\nClassification Report\n")
# target_names = [key for (key, value) in selected_beats.items()]

pd.set_option("display.max_rows", 6000)
pd.set_option("display.max_columns",200)
pd.set_option('display.width', 1000)

print(classification_report(y_validate_mod, y_predicted_mod, target_names=class_names, digits=3))
report = classification_report(y_validate_mod, y_predicted_mod, target_names=class_names, output_dict=True)
# output_dictbool, default=False, If True, return output as dict.
df_report = pd.DataFrame(report).transpose()
# https://medium.com/@asmaiya/you-can-something-like-this-84d28e0fd31f

# Įrašome į diską
filepath = Path(path_for_results, 'apibendrinti_rezultatai.csv') 
df_report.to_csv(filepath)    
print(f'\nApibendrinti_rezultatai įrašyti į:  {filepath}')



MODELIO TIKSLUMO VERTINIMO REZULTATAI
Modelis iš aplanko:  model_cnn_fda_vu_v1
Pūpsnių klasės:  ['N', 'S', 'V', 'U'] [845  20   0   0] Total: 865
Klasifikuojamos klasės: ['N', 'S', 'V']

APIBENDRINTI REZULTATAI

Confusion Matrix
     N   S   V
N  802  38   3
S    3   5  12
V    0   0   0


Zero values! Cannot calculate Normalized Confusion Matrix

Classification Report

              precision    recall  f1-score   support

           N      0.996     0.951     0.973       843
           S      0.116     0.250     0.159        20
           V      0.000     0.000     0.000         0

    accuracy                          0.935       863
   macro avg      0.371     0.400     0.377       863
weighted avg      0.976     0.935     0.954       863


Apibendrinti_rezultatai įrašyti į:  /home/kesju/DI/DUOM_2022_RUDUO_2/rezultatai_tst/apibendrinti_rezultatai.csv
